In [ ]:
from dotenv import load_dotenv
from IPython.display import display, Markdown
import re
import os
from google.colab import userdata
from mistralai import Mistral

In [ ]:
# Access the secret
api_key = userdata.get('MISTRAL_API_KEY')

if not api_key:
    raise ValueError("❌ MISTRAL_API_KEY not found in Colab secrets!")
else:
    print("✅ API key loaded successfully from Colab secrets!")

# Initialize Mistral client
client_1 = Mistral(api_key=api_key)

# Test connection
def test_connection():
    try:
        models = client.models.list()
        print("✅ Connected successfully!")
        print(f"Available models: {[m.id for m in models.data]}")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        print("💡 If key is not active yet, wait a few minutes and try again")

test_connection()

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.find('tokenizers/punkt_tab')

In [ ]:
!wget https://raw.githubusercontent.com/langchain-ai/langchain/9a277cbe007706b48fa98787ab85a11c59ccba2e/docs/docs/integrations/document_loaders/example_data/layout-parser-paper.pdf

In [ ]:
import nltk

from pathlib import Path
from pprint import pprint

from langchain.document_loaders import PyPDFLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive connected.")

In [ ]:
PDF_FILE_PATH = "/content/drive/MyDrive/MulRAG.pdf"

In [ ]:
loader = PyPDFLoader(PDF_FILE_PATH, extract_images=True)
pages = loader.load()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pages

In [ ]:
len(pages)

In [ ]:
WEAVIATE_API_KEY = "UU02aHp5NFlBMnVHSWRSTV9LakoyNGY1dzJmQzVsaExKYzR5S0VzeVo0TWtxNExyZGZnek5BU0swU3ZZPV92MjAw"
WEAVIATE_CLUSTER = "hhnfg8g8ruijh30yl7jya.c0.europe-west3.gcp.weaviate.cloud"

In [ ]:
import weaviate
from weaviate.auth import AuthApiKey

WEAVIATE_URL = "https://" + WEAVIATE_CLUSTER
auth = AuthApiKey(api_key=WEAVIATE_API_KEY)
client = weaviate.Client(
    url=WEAVIATE_URL,
    auth_client_secret=auth
)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# подготовка к векторизации (будем использовать в Weaviate.from_documents)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [ ]:
# Разделение текста на чанки (chunks)
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Создание экземпляра разделителя текста
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Размер каждого чанка в символах
    chunk_overlap=20   # Перекрытие между чанками в символах
)

# Применение разделителя к документам (pages)
docs = text_splitter.split_documents(pages)

In [ ]:
docs

In [ ]:
# ругалось на формат ptex_fullbanner, скорее всего если документов будет много будет плохо(долго) работать
for doc in docs:
    if 'ptex.fullbanner' in doc.metadata:
        doc.metadata['ptex_fullbanner'] = doc.metadata.pop('ptex.fullbanner')


In [ ]:
  # Измерим метрики после сохранения в Weaviate и выберем оптимальные размеры чанков(точность поиска (precision/recall), время запросов, релевантность ответов.)
  #from langchain.text_splitter import RecursiveCharacterTextSplitter

  # Тестируем разные размеры
  #sizes = [200, 500, 1000]
  #for size in sizes:
      #text_splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=int(size*0.1))
      #docs = text_splitter.split_documents(pages)
      # Далее: векторизация и оценка
      #print(f"Размер {size}: {len(docs)} чанков")

In [ ]:
# создание экземпляра векторного хранилища на основе наших документов
from langchain.vectorstores import Weaviate
vector_db = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)

In [ ]:
vector_db.similarity_search("what is rag?", k=3)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
!pip install -qU langchain-mistralai

In [ ]:
from langchain_mistralai import ChatMistralAI

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
model = ChatMistralAI(
    api_key=api_key,
    model="mistral-large-latest",
    temperature=0.7,
    max_tokens=512
)


In [ ]:
output_parser = StrOutputParser() #для чистого вывода ответа
retriever = vector_db.as_retriever() # поисковик на основе наших документов

In [ ]:
rag_chain = (
    {"context": vector_db.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [ ]:
rag_chain.invoke("what is Multimodal RAG?")

In [ ]:
rag_chain.invoke("Retell the summary of the article about multimodul RAG")

In [ ]:
retriever.invoke("What is Multimodal RAG?")

In [ ]:
import re
import cv2
import pytesseract
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

In [ ]:
class Ocr:
    def preprocess_image(self, image_path, scale_percent=200):
        self.scale_percent = scale_percent
        img = cv2.imread(image_path)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        width = int(gray_img.shape[1] * self.scale_percent / 100)
        height = int(gray_img.shape[0] * self.scale_percent / 100)
        gray_img = cv2.resize(gray_img, (width, height), interpolation=cv2.INTER_CUBIC)
        denoised = cv2.fastNlMeansDenoising(gray_img, h=10)
        return denoised

    def clean_text(self, text):
        cleaned = re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9\s.,;:!?()%\-—–«»]', '', text)
        return cleaned

    def img2txt(self, img, min_confidence=60):
        self.min_confidence = min_confidence
        processed = self.preprocess_image(img)
        config = '--psm 6 --oem 3'
        data = pytesseract.image_to_data(
            processed,
            lang='rus+eng',
            config=config,
            output_type=pytesseract.Output.DICT
        )
        filtered_text = []
        for i, conf in enumerate(data['conf']):
            if int(conf) > self.min_confidence:
                filtered_text.append(data['text'][i])
        return ' '.join(filtered_text)

    def process(self, image_path):
        text = self.img2txt(image_path)
        return self.clean_text(text)


class ImageCaptioner:
    def __init__(self, model_name="Salesforce/blip-image-captioning-large"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.processor = BlipProcessor.from_pretrained(model_name)
        self.model = BlipForConditionalGeneration.from_pretrained(model_name).to(self.device)

    def describe(self, image_path, max_length=50):
        image = Image.open(image_path)
        inputs = self.processor(images=image, return_tensors="pt").to(self.device)

        output = self.model.generate(**inputs, max_new_tokens=max_length)
        caption = self.processor.decode(output[0], skip_special_tokens=True)

        return caption

    def describe_with_context(self, image_path, question):
        image = Image.open(image_path)
        inputs = self.processor(images=image, text=question, return_tensors="pt").to(self.device)

        output = self.model.generate(**inputs)
        answer = self.processor.decode(output[0], skip_special_tokens=True)

        return answer

In [ ]:
captioner = ImageCaptioner()
description = captioner.describe("The_life_of_Tolstoy.jpg")
print(description)


ocr = Ocr()
text = ocr.process('The_life_of_Tolstoy.jpg')
print(text)